In [ ]:
##CS404 Artifical Intelligence Project
##Supplementary code to go with paper
##Adnan Silajdzic
##Edin Ziga
##Mirza Redzepovic

In [ ]:
!pip install pillow
!pip install tqdm
!pip install numpy
!pip install tensorflow
!pip install pydot
!pip install pydot_ng 

In [1]:
import os
import numpy as np
import pydot
import graphviz
import random

from PIL import Image
from tqdm import tqdm

#TensorFlow implementation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model

In [2]:
#LOAD IMAGES

# Parameters
imageSize = 48
# Set the desired size (in this case, 28x28)
size = (imageSize, imageSize)

#Directory of chest_xray\train\NORMAL
directory = r"D:\PythonProjects\PneumoninaANN\chest_xray\train\NORMAL"

NormalTraining = []
for filename in tqdm(os.listdir(directory), desc = "Loading Normal Training Data"):
    # Load the image
    image = Image.open(os.path.join(directory, filename))
    image = image.convert('L')
    image = image.resize(size)
    pixels = list(image.getdata())
    pixels_normalized = [pixel/255.0 for pixel in pixels]
    NormalTraining.append(pixels_normalized)
    
#Directory of chest_xray\train\PNEUMONIA
directory = r"D:\PythonProjects\PneumoninaANN\chest_xray\train\PNEUMONIA"

PneumoniaTraining = []
for filename in tqdm(os.listdir(directory), desc = "Loading Pneumonia Training Data"):
    image = Image.open(os.path.join(directory, filename))
    image = image.convert('L')
    image = image.resize(size)
    pixels = list(image.getdata())
    pixels_normalized = [pixel/255.0 for pixel in pixels]
    PneumoniaTraining.append(pixels_normalized)

#Directory of chest_xray\test\NORMAL
directory = r"D:\PythonProjects\PneumoninaANN\chest_xray\test\NORMAL"  
    
NormalTesting = []
for filename in tqdm(os.listdir(directory), desc = "Loading Normal Testing Data"):
    image = Image.open(os.path.join(directory, filename))
    image = image.convert('L')
    image = image.resize(size)
    pixels = list(image.getdata())
    pixels_normalized = [pixel/255.0 for pixel in pixels]
    NormalTesting.append(pixels_normalized)

#Directory of chest_xray\test\PNEUMONIA
directory = r"D:\PythonProjects\PneumoninaANN\chest_xray\test\PNEUMONIA"
    
PneumoniaTesting = []
for filename in tqdm(os.listdir(directory), desc = "Loading Pneumonia Testing Data"):
    # Load the image
    image = Image.open(os.path.join(directory, filename))
    image = image.convert('L')
    image = image.resize(size)
    pixels = list(image.getdata())
    pixels_normalized = [pixel/255.0 for pixel in pixels]
    PneumoniaTesting.append(pixels_normalized)
    
print("Images successfully loaded")
print(f"All images are grayscaled and are {imageSize} by {imageSize}")

training_data = NormalTraining + PneumoniaTraining
testing_data = NormalTesting + PneumoniaTesting

training_labels = [0]*len(NormalTraining) + [1]*len(PneumoniaTraining)
testing_labels = [0]*len(NormalTesting) + [1]*len(PneumoniaTesting)

print(f"Training data size - {len(training_data)}")
print(f"Testing data size - {len(testing_data)}")

Loading Pneumonia Testing Data: 100%|███████████████████████████████████████████████| 390/390 [00:01<00:00, 231.01it/s]

Images successfully loaded
All images are grayscaled and are 48 by 48
Training data size - 5216
Testing data size - 624


In [11]:
#RANDOM SAMPLING WITH REPLACEMENT
print(f'Training data full size - {len(training_data)}')
print(f'Testing data full size - {len(testing_data)}')

#Parameters
trainingDataSize = 1800
testingDataSize = 200
noEpochs = 15

print(f'Training data sample size - {trainingDataSize}')
print(f'Testing data sample size - {testingDataSize}')
print(f'No. epochs selected - {noEpochs}')

training_data_sample=[]
training_labels_sample=[]

testing_data_sample=[]
testing_labels_sample=[]

for i in tqdm(range(trainingDataSize), desc = "Loading Training Data"):
    randNum = random.randint(0, 1)
    if randNum == 0:
        randData = random.randint(0, len(NormalTraining)-1)
        training_data_sample.append(NormalTraining[randData])
        training_labels_sample.append(0)
    else:
        randData = random.randint(0, len(PneumoniaTraining)-1)
        training_data_sample.append(PneumoniaTraining[randData])
        training_labels_sample.append(1)
        
        
for i in tqdm(range(testingDataSize), desc = "Loading Testing Data"):
    randNum = random.randint(0, 1)
    if randNum == 0:
        randData = random.randint(0, len(NormalTesting)-1)
        testing_data_sample.append(NormalTesting[randData])
        testing_labels_sample.append(0)
    else:
        randData = random.randint(0, len(PneumoniaTesting)-1)
        testing_data_sample.append(PneumoniaTesting[randData])
        testing_labels_sample.append(1)


np.random.seed(42)

model = Sequential()

model.add(Dense(1024, activation='tanh', input_dim=imageSize*imageSize))
model.add(Dense(512, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Loading Model')
model.fit(training_data_sample, training_labels_sample, epochs=noEpochs, batch_size=64)

test_loss, test_accuracy = model.evaluate(testing_data_sample, testing_labels_sample)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Training data full size - 5216
Testing data full size - 624
Training data sample size - 1800
Testing data sample size - 200
No. epochs selected - 15


Loading Testing Data: 100%|██████████████████████████████████████████████████████████████████| 200/200 [00:00<?, ?it/s]

Loading Model


Epoch 1/15
29/29 [==============================] - 2s 34ms/step - loss: 0.8722 - accuracy: 0.5261
Epoch 2/15
29/29 [==============================] - 1s 35ms/step - loss: 0.5689 - accuracy: 0.7078
Epoch 3/15
29/29 [==============================] - 1s 33ms/step - loss: 0.4897 - accuracy: 0.7817
Epoch 4/15
29/29 [==============================] - 1s 34ms/step - loss: 0.4026 - accuracy: 0.8306
Epoch 5/15
29/29 [==============================] - 1s 33ms/step - loss: 0.2920 - accuracy: 0.8817
Epoch 6/15
29/29 [==============================] - 1s 34ms/step - loss: 0.2441 - accuracy: 0.9089
Epoch 7/15
29/29 [==============================] - 1s 32ms/step - loss: 0.2161 - accuracy: 0.9178
Epoch 8/15
29/29 [==============================] - 1s 33ms/step - loss: 0.1713 - accuracy: 0.9333
Epoch 9/15
29/29 [==============================] - 1s 33ms/step - loss: 0.1814 - accuracy: 0.9372
Epoch 10/15
29/29 [==============================] - 1s 33ms/step - loss: 0.1728 - accuracy: 0.9339
Epoch 11/

In [10]:
#RANDOM SAMPLING WITHOUT REPLACEMENT
print(f'Training data full size - {len(training_data)}')
print(f'Testing data full size - {len(testing_data)}')

#Parameters
trainingDataSize = 1800
testingDataSize = 200
noEpochs = 15

print(f'Training data sample size - {trainingDataSize}')
print(f'Testing data sample size - {testingDataSize}')
print(f'No. epochs selected - {noEpochs}')

training_data_sample=[]
training_labels_sample=[]

testing_data_sample=[]
testing_labels_sample=[]

# Training data
normal_indices = list(range(len(NormalTraining)))
pneumonia_indices = list(range(len(PneumoniaTraining)))

for i in tqdm(range(trainingDataSize), desc="Loading Training Data"):
    randNum = random.randint(0, 1)
    if randNum == 0 and len(normal_indices) > 0:
        randData = random.sample(normal_indices, 1)[0]
        normal_indices.remove(randData)
        training_data_sample.append(NormalTraining[randData])
        training_labels_sample.append(0)
    elif randNum == 1 and len(pneumonia_indices) > 0:
        randData = random.sample(pneumonia_indices, 1)[0]
        pneumonia_indices.remove(randData)
        training_data_sample.append(PneumoniaTraining[randData])
        training_labels_sample.append(1)
        
# Testing data
normal_indices = list(range(len(NormalTesting)))
pneumonia_indices = list(range(len(PneumoniaTesting)))

for i in tqdm(range(testingDataSize), desc="Loading Testing Data"):
    randNum = random.randint(0, 1)
    if randNum == 0 and len(normal_indices) > 0:
        randData = random.sample(normal_indices, 1)[0]
        normal_indices.remove(randData)
        testing_data_sample.append(NormalTesting[randData])
        testing_labels_sample.append(0)
    elif randNum == 1 and len(pneumonia_indices) > 0:
        randData = random.sample(pneumonia_indices, 1)[0]
        pneumonia_indices.remove(randData)
        testing_data_sample.append(PneumoniaTesting[randData])
        testing_labels_sample.append(1)


np.random.seed(42)

model = Sequential()

model.add(Dense(1024, activation='tanh', input_dim=imageSize*imageSize))
model.add(Dense(512, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Loading Model')
model.fit(training_data_sample, training_labels_sample, epochs=noEpochs, batch_size=64)

test_loss, test_accuracy = model.evaluate(testing_data_sample, testing_labels_sample)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Training data full size - 5216
Testing data full size - 624
Training data sample size - 1800
Testing data sample size - 200
No. epochs selected - 15


Loading Testing Data: 100%|██████████████████████████████████████████████████████| 200/200 [00:00<00:00, 100066.90it/s]

Loading Model


Epoch 1/15
29/29 [==============================] - 2s 36ms/step - loss: 0.9525 - accuracy: 0.4950
Epoch 2/15
29/29 [==============================] - 1s 32ms/step - loss: 0.6582 - accuracy: 0.6289
Epoch 3/15
29/29 [==============================] - 1s 38ms/step - loss: 0.4004 - accuracy: 0.8328
Epoch 4/15
29/29 [==============================] - 1s 34ms/step - loss: 0.3071 - accuracy: 0.8778
Epoch 5/15
29/29 [==============================] - 1s 36ms/step - loss: 0.2547 - accuracy: 0.9006
Epoch 6/15
29/29 [==============================] - 1s 34ms/step - loss: 0.1839 - accuracy: 0.9289
Epoch 7/15
29/29 [==============================] - 1s 38ms/step - loss: 0.2539 - accuracy: 0.8967
Epoch 8/15
29/29 [==============================] - 1s 34ms/step - loss: 0.1819 - accuracy: 0.9350
Epoch 9/15
29/29 [==============================] - 1s 35ms/step - loss: 0.1527 - accuracy: 0.9461
Epoch 10/15
29/29 [==============================] - 1s 35ms/step - loss: 0.1747 - accuracy: 0.9350
Epoch 11/